In [ ]:
#!conda install pyshp

All data cleaned and put into a useable format using ArcGIS. I can write up documentatin for that later

In [3]:
#import shapefile 
import numpy as np
import pandas as pd
import sklearn.model_selection
import sklearn.linear_model

blargh write some documentation

In [4]:
predictors = pd.read_csv('pnw_predictors.txt')
carbon = pd.read_csv('pnw_c.txt')

In [5]:
full_data = pd.merge(predictors, carbon, how='left', left_on=['X','Y'], right_on=['X', 'Y'])
or_sequestration = full_data[['raster_OR_c']].dropna()['raster_OR_c']
wa_sequestration = full_data[['raster_WA_c']].dropna()['raster_WA_c']
full_sequestration = list(or_sequestration)+list(wa_sequestration)
full_data['raster_OR_c'].fillna(full_data['raster_WA_c'], inplace=True)

cleaner = full_data.drop(['X', 'Y', 'OBJECTID_x', 'OR_random_Merge_x', 'OBJECTID_y', 'OR_random_Merge_y', 'raster_WA_c'], axis=1)
cleaner = cleaner.dropna()

In [6]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(cleaner.iloc[:,0:23], cleaner.iloc[:,23], test_size=0.2)

In [7]:
model = sklearn.linear_model.Lasso()
model.fit(X_train, y_train)
model.score(X_train, y_train)

0.14913333146448338

In [8]:
X_train.corr()

,wc2_0_bio_10m_02,wc2_0_bio_10m_03,wc2_0_bio_10m_04,wc2_0_bio_10m_05,wc2_0_bio_10m_06,wc2_0_bio_10m_07,wc2_0_bio_10m_08,wc2_0_bio_10m_09,wc2_0_bio_10m_10,wc2_0_bio_10m_11,...,wc2_0_bio_10m_15,wc2_0_bio_10m_16,wc2_0_bio_10m_17,wc2_0_bio_10m_18,wc2_0_bio_10m_19,Bare_ground_change_1982_2016__1_,Short_vegetation_change_1982_2016,Tree_canopy_change_1982_2016,gt30w140n90,wc2_0_bio_10m_01
wc2_0_bio_10m_02,1.000000,0.444146,0.537304,0.742494,-0.553058,0.818639,-0.156054,0.456175,0.397764,-0.234167,...,-0.399237,-0.734727,-0.812246,-0.811011,-0.730410,-0.021847,0.137278,-0.080420,0.373196,0.027322
wc2_0_bio_10m_03,0.444146,1.000000,-0.493941,0.334818,0.405522,-0.139680,0.472508,0.309932,0.263183,0.613383,...,0.332139,0.077584,-0.364557,-0.390059,0.084317,-0.078754,-0.020773,0.068633,-0.087840,0.516921
wc2_0_bio_10m_04,0.537304,-0.493941,1.000000,0.421818,-0.900710,0.920765,-0.567325,0.164424,0.181557,-0.790514,...,-0.720502,-0.821542,-0.507092,-0.468727,-0.821989,0.076610,0.166541,-0.172324,0.427666,-0.429365
wc2_0_bio_10m_05,0.742494,0.334818,0.421818,1.000000,-0.149825,0.640239,0.255257,0.890640,0.897003,0.177302,...,-0.265661,-0.540231,-0.705540,-0.675001,-0.536434,0.097162,0.113276,-0.125790,-0.154700,0.564125
wc2_0_bio_10m_06,-0.553058,0.405522,-0.900710,-0.149825,1.000000,-0.855429,0.747322,0.164707,0.190759,0.932375,...,0.679045,0.769234,0.427242,0.415361,0.769708,0.013209,-0.146239,0.107256,-0.664876,0.714345
wc2_0_bio_10m_07,0.818639,-0.139680,0.920765,0.640239,-0.855429,1.000000,-0.446916,0.338579,0.321671,-0.631528,...,-0.666743,-0.880644,-0.701529,-0.676301,-0.879024,0.040634,0.172958,-0.149225,0.435535,-0.259500
wc2_0_bio_10m_08,-0.156054,0.472508,-0.567325,0.255257,0.747322,-0.446916,1.000000,0.454560,0.490522,0.818875,...,0.371070,0.399212,0.089534,0.107052,0.399663,0.074406,-0.069031,0.018562,-0.553894,0.782225
wc2_0_bio_10m_09,0.456175,0.309932,0.164424,0.890640,0.164707,0.338579,0.454560,1.000000,0.942499,0.439544,...,-0.011483,-0.251663,-0.482121,-0.463996,-0.249672,0.131628,0.069863,-0.112962,-0.402874,0.763706
wc2_0_bio_10m_10,0.397764,0.263183,0.181557,0.897003,0.190759,0.321671,0.490522,0.942499,1.000000,0.457467,...,-0.039581,-0.243093,-0.474427,-0.429599,-0.240024,0.142754,0.072528,-0.122316,-0.475178,0.807510
wc2_0_bio_10m_11,-0.234167,0.613383,-0.790514,0.177302,0.932375,-0.631528,0.818875,0.439544,0.457467,1.000000,...,0.627341,0.588791,0.160627,0.153366,0.590815,0.017392,-0.105394,0.080684,-0.672188,0.888311


## K-fold validation set

In [12]:
from sklearn.model_selection import KFold

def compute_CV_error(model, X_train, Y_train):
    '''
    Split the training data into 4 subsets.
    For each subset, 
        fit a model holding out that subset
        compute the MSE on that subset (the validation set)
    You should be fitting 4 models total.
    Return the average MSE of these 4 folds.

    Args:
        model: an sklearn model with fit and predict functions 
        X_train (data_frame): Training data
        Y_train (data_frame): Label 

    Return:
        the average validation MSE for the 4 splits.
    '''
    kf = KFold(n_splits=4)
    validation_errors = []
    
    for train_idx, valid_idx in kf.split(X_train):
        # split the data
        split_X_train, split_X_valid = X_train.iloc[train_idx], X_train.iloc[valid_idx]
        split_Y_train, split_Y_valid = Y_train.iloc[train_idx], Y_train.iloc[valid_idx]

        # Fit the model on the training split
        model.fit(split_X_train, split_Y_train)
        
        # Compute the RMSE on the validation split
        error = model.score()

        # YOUR CODE HERE
        #raise NotImplementedError()
        
        validation_errors.append(error)
        
    return np.mean(validation_errors)

In [13]:
compute_CV_error(model, X_train, y_train)

NameError: name 'rmse' is not defined